# Easy data access and manipulation using *xray*
----

*xray* is a library which allows you to access multi-dimensional, labelled
data in a easy manner.

It makes most calculations we do in our day-to-day work a matter of a few lines.

*xray* depends on netCDF4 (discussed last time) to , so install that before!

In [1]:
import xray

dataset = xray.open_dataset('/data/MonthlyFields/uWinds.nc')

*xray* allows you to use labels to access data instead of numerical indices. This makes your analysis much easier to read for a person who has not seen it before.

## Two kinds of objects in *xray*

*xray* contains two kinds of objects: **Dataset** and **DataArray**

**Dataset** represents an open file, while **DataArray** represents the data extracted
from a file.

In [2]:
print dataset

<xray.Dataset>
Dimensions:    (latitude: 241, level: 6, longitude: 480, time: 432)
Coordinates:
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 6.75 7.5 8.25 ...
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 87.0 86.25 85.5 84.75 84.0 83.25 ...
  * level      (level) int32 100 150 200 300 500 700
  * time       (time) datetime64[ns] 1979-01-01 1979-02-01 1979-03-01 1979-04-01 ...
Data variables:
    u          (time, level, latitude, longitude) float64 -9.486 -9.621 -9.756 -9.889 ...
Attributes:
    Conventions: CF-1.0
    history: 2015-01-23 03:06:51 GMT by grib_to_netcdf-1.12.3: grib_to_netcdf /data/data01/netcdf-web221-20150123030414-21936-4167.target -o /data/data01/netcdf-web221-20150123030641-21936-4168.nc


We can also access each variable in the file by its name...

In [4]:
winds = dataset.u
print winds

<xray.DataArray 'u' (time: 432, level: 6, latitude: 241, longitude: 480)>
[299842560 values with dtype=float64]
Coordinates:
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 87.0 86.25 85.5 84.75 84.0 83.25 ...
  * time       (time) datetime64[ns] 1979-01-01 1979-02-01 1979-03-01 1979-04-01 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 6.75 7.5 8.25 ...
  * level      (level) int32 100 150 200 300 500 700
Attributes:
    units: m s**-1
    long_name: U component of wind
    standard_name: eastward_wind


We can then read data using labels...

In [7]:
winds.loc['1980-3-5':'1982-4-4']

<xray.DataArray 'u' (time: 25, level: 6, latitude: 241, longitude: 480)>
[17352000 values with dtype=float64]
Coordinates:
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 87.0 86.25 85.5 84.75 84.0 83.25 ...
  * time       (time) datetime64[ns] 1980-04-01 1980-05-01 1980-06-01 1980-07-01 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 6.75 7.5 8.25 ...
  * level      (level) int32 100 150 200 300 500 700
Attributes:
    units: m s**-1
    long_name: U component of wind
    standard_name: eastward_wind

to select more than one dimension, send in a dictionary...

In [9]:
selection = {}
selection['time'] = slice('1980-3-5','1982-4-4')
selection['latitude'] = slice(40,-40)
selection['longitude'] = slice(30,100)
selection['level'] = slice(200,500)

winds.loc[selection]

<xray.DataArray 'u' (time: 25, level: 3, latitude: 107, longitude: 94)>
[754350 values with dtype=float64]
Coordinates:
  * latitude   (latitude) float32 39.75 39.0 38.25 37.5 36.75 36.0 35.25 34.5 33.75 33.0 ...
  * time       (time) datetime64[ns] 1980-04-01 1980-05-01 1980-06-01 1980-07-01 ...
  * longitude  (longitude) float32 30.0 30.75 31.5 32.25 33.0 33.75 34.5 35.25 36.0 36.75 ...
  * level      (level) int32 200 300 500
Attributes:
    units: m s**-1
    long_name: U component of wind
    standard_name: eastward_wind

Writing to an nc file is also easy! If it is a **DataArray**, we first convert it to a **Dataset** and then write to an nc file.

In [12]:
winds.loc[selection].to_dataset().to_netcdf('/home/joymm/test.nc')

## directly available functions

Both **Dataset** and **DataArray** support the basic functions:

* mean
* variance
* standard deviation
* product
* sum
* press winds.[TAB] to see all!

All basic functions take an optional argument specifying which coordinate
to work over.

## groupby

This is a powerful function, which allows you to use the basic
functions in a more sophisticated manner.

For example, if we want to calculate monthly climatology,

In [15]:
clim = winds.groupby('time.month').mean(dim='time')

In [20]:
clim.coords

Coordinates:
  * level      (level) int32 100 150 200 300 500 700
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 6.75 7.5 8.25 ...
  * month      (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 87.0 86.25 85.5 84.75 84.0 83.25 ...

And then easily calculate monthly anomalies

In [21]:
monthlyAnom = winds.groupby('time.month') - clim

In [23]:
print monthlyAnom.coords

Coordinates:
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 87.0 86.25 85.5 84.75 84.0 83.25 ...
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 6.75 7.5 8.25 ...
  * level      (level) int32 100 150 200 300 500 700
  * time       (time) datetime64[ns] 1979-01-01 1979-02-01 1979-03-01 1979-04-01 ...
    month      (time) int32 1 2 3 4 5 6 7 8 9 10 11 12 1 2 3 4 5 6 7 8 9 10 11 12 1 2 3 4 5 6 ...


You can also easily "resample" the data, take averages of arbitrary periods...

In [27]:
quarterly = winds.resample('Q-SEP',dim='time')

In [29]:
quarterly.coords

Coordinates:
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 87.0 86.25 85.5 84.75 84.0 83.25 ...
  * level      (level) int32 100 150 200 300 500 700
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 6.75 7.5 8.25 ...
  * time       (time) datetime64[ns] 1979-03-31 1979-06-30 1979-09-30 1979-12-31 ...

And also easily calculate seasonal averages (season is defined as 3 months in xray)

In [32]:
seasonalMean = winds.groupby('time.season').mean(dim='time')
print seasonalMean.coords, seasonalMean.shape

Coordinates:
  * level      (level) int32 100 150 200 300 500 700
  * season     (season) object 'DJF' 'JJA' 'MAM' 'SON'
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 3.0 3.75 4.5 5.25 6.0 6.75 7.5 8.25 ...
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 87.0 86.25 85.5 84.75 84.0 83.25 ... (4, 6, 241, 480)


In [36]:
oneLevel = winds.sel(level=200)